In [1]:
import numpy as np
import os
import time
import torch

In [2]:
print(np.__version__)
print(torch.__version__)

1.18.1
1.5.0


# Vectorization

In [3]:
def basic_dot(x, y):
    assert(len(x) == len(y))
    
    result = 0
    for i in range(len(x)):
        result += x[i]*y[i]
        
    return result

In [4]:
def dot(x, y):
    return np.dot(x, y)

In [5]:
np.random.seed(0)
X = np.random.randint(-1000, 1000, size=3000)
Y = np.random.randint(-1000, 1000, size=3000)

print(dot(X,Y))
assert basic_dot(X,Y) == dot(X,Y)

7082791


In [6]:
def basic_outer(x, y):
    result = np.zeros((len(x), len(y))) 
    for i in range(len(x)):
        for j in range(len(y)):
            result[i, j] = x[i]*y[j]
    
    return result

In [7]:
def outer(x, y):
    return np.outer(x, y)

In [8]:
np.random.seed(0)
X = np.random.randint(-1000, 1000, size=3000)
Y = np.random.randint(-1000, 1000, size=3000)

print(outer(X,Y))
assert np.all(basic_outer(X,Y) == outer(X,Y))

[[  59092 -144096  136512 ...  -53088  -86268   53404]
 [  82467 -201096  190512 ...  -74088 -120393   74529]
 [-122111  297768 -282096 ...  109704  178269 -110357]
 ...
 [-144551  352488 -333936 ...  129864  211029 -130637]
 [-179707  438216 -415152 ...  161448  262353 -162409]
 [  88825 -216600  205200 ...  -79800 -129675   80275]]


In [9]:
def basic_multiply(x, y):
    assert(len(x) == len(y))
    
    result = np.zeros(len(x))
    for i in range(len(x)):
        result[i] = x[i]*y[i]
    
    return result

In [10]:
def multiply(x, y):
    return np.multiply(x, y)

In [11]:
np.random.seed(0)
X = np.random.randint(-1000, 1000, size=3000)
Y = np.random.randint(-1000, 1000, size=3000)

print(multiply(X,Y))
assert np.all(basic_multiply(X,Y) == multiply(X,Y))

[  59092 -201096 -282096 ...  129864  262353   80275]


In [12]:
def basic_sumproduct(x, y):
    assert(len(x) == len(y))
    
    result = 0
    for i in range(len(x)):
        for j in range(len(y)):
            result += x[i] * y[j]
            
    return result

In [13]:
def sumproduct(x, y):
    return np.sum(np.sum(x) * y)

In [14]:
np.random.seed(0)
X = np.random.randint(-1000, 1000, size=3000)
Y = np.random.randint(-1000, 1000, size=3000)

print(sumproduct(X,Y))
assert basic_sumproduct(X,Y) == sumproduct(X,Y)

265421520


In [15]:
def basic_ReLU(x):
    result = np.copy(x)
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            if x[i][j] < 0:
                result[i][j] = 0
                
    return result

In [23]:
def ReLU(x):
    return np.where(x >= 0, x, 0)

In [24]:
np.random.seed(0)
X = np.random.randint(-1000, 1000, size=(3000,3000))

print(ReLU(X))
assert np.all(basic_ReLU(X) == ReLU(X))

[[  0   0 653 ... 773 961   0]
 [  0 456   0 ... 168 273   0]
 [936 475   0 ... 408   0   0]
 ...
 [  0 396 457 ... 646   0   0]
 [645 943   0 ... 863   0 790]
 [641   0 379 ... 347   0   0]]


In [25]:
def basic_PrimeReLU(x):
    result = np.copy(x)
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            if x[i][j] < 0:
                result[i][j] = 0
            else:
                result[i][j] = 1
                
    return result

In [26]:
def PrimeReLU(x):
    return np.where(x >= 0, 1, 0)

In [27]:
np.random.seed(0)
X = np.random.randint(-1000, 1000, size=(3000,3000))

print(PrimeReLU(X))
print(basic_PrimeReLU(X))
assert np.all(basic_PrimeReLU(X) == PrimeReLU(X))

[[0 0 1 ... 1 1 0]
 [0 1 0 ... 1 1 0]
 [1 1 0 ... 1 0 0]
 ...
 [0 1 1 ... 1 0 0]
 [1 1 0 ... 1 0 1]
 [1 0 1 ... 1 0 0]]
[[0 0 1 ... 1 1 0]
 [0 1 0 ... 1 1 0]
 [1 1 0 ... 1 0 0]
 ...
 [0 1 1 ... 1 0 0]
 [1 1 0 ... 1 0 1]
 [1 0 1 ... 1 0 0]]


# Preprocessing

## Slicing Last Point

In [28]:
def get_data_1():
    """
    This is the generating process from which example data 1 will derive
    
    Parameters: 
    None
    
    Returns: 
    numpy.ndarray: 1-d numpy array with 2-d numpy arrays as elements.
    """
    freq000 = 3; freq001 = 1; freq002 = 4; freq003 = 1
    freq010 = 5; freq011 = 9; freq012 = 2; freq013 = 6
    freq020 = 5; freq021 = 3; freq022 = 5; freq023 = 8
    frame00 = np.array([freq000, freq001, freq002, freq003])
    frame01 = np.array([freq010, freq011, freq012, freq013])
    frame02 = np.array([freq020, freq021, freq022, freq023])
    utterance0 = np.array([frame00, frame01, frame02])

    freq100 = 9; freq101 = 7; freq102 = 9; freq103 = 3
    freq110 = 2; freq111 = 3; freq112 = 8; freq113 = 4
    frame10 = np.array([freq100, freq101, freq102, freq103])
    frame11 = np.array([freq110, freq111, freq112, freq113])
    utterance1 = np.array([frame10, frame11])

    freq200 = 6; freq201 = 2; freq202 = 6; freq203 = 4
    freq210 = 3; freq211 = 3; freq212 = 8; freq213 = 3
    freq220 = 2; freq221 = 7; freq222 = 9; freq223 = 5
    freq230 = 0; freq231 = 2; freq232 = 8; freq233 = 8
    frame20 = np.array([freq200, freq201, freq202, freq203])
    frame21 = np.array([freq210, freq211, freq212, freq213])
    frame22 = np.array([freq220, freq221, freq222, freq223])
    frame23 = np.array([freq230, freq231, freq232, freq233])
    utterance2 = np.array([frame20, frame21, frame22, frame23])

    spectrograms = np.array([utterance0, utterance1, utterance2])

    return spectrograms

def get_data_2():
    """
    This is the generating process from which example data 2 will derive
    
    Parameters: 
    None
    
    Returns: 
    numpy.ndarray: 1-d numpy array with 2-d numpy arrays as elements.
    """
    np.random.seed(0)
    recordings = np.random.randint(10)
    durations = [np.random.randint(low=5, high=10) 
                 for i in range(recordings)]
    data = []
    k = 40 # Given as fixed constant
    for duration in durations:
        data.append(np.random.randint(10, size=(duration, k)))
    data = np.asarray(data)
    return data

In [38]:
def slice_last_point(x, m):
    """
    Takes one 3-dimensional array with the length of the output instances.
    Your task is to keep only the last m points for each instances in 
    the dataset.

    Parameters: 
    x (numpy.ndarray): 1-d numpy array with 2-d numpy arrays as elements (n, ?, k). 
    m (int): The cutoff reference index in dimension 2.
  
    Returns: 
    numpy.ndarray: A 3-dimensional numpy array of shape (n, m, k)
    """
    spectrograms = x
    
    # Input function dimension specification
    assert(spectrograms.ndim == 1)
    for utter in spectrograms:
        assert(utter.ndim == 2)

    # Pre-define output function dimension specification
    dim1 = spectrograms.shape[0]    # n
    dim2 = m                       # m
    dim3 = spectrograms[0].shape[1] # k

    result = np.zeros((dim1,dim2,dim3))

    #### Start of your code ####
    for i in range(dim1):
        result[i,:,:] = x[i][-m:, :]
    
    ####  End of your code  ####

    # Assert output function dimension specification
    assert(result.shape[0] == dim1)
    assert(result.shape[1] == dim2)
    assert(result.shape[2] == dim3)
    
    return result

In [39]:
spectrograms = get_data_1()
duration = 2
print(slice_last_point(spectrograms, duration))

[[[5. 9. 2. 6.]
  [5. 3. 5. 8.]]

 [[9. 7. 9. 3.]
  [2. 3. 8. 4.]]

 [[2. 7. 9. 5.]
  [0. 2. 8. 8.]]]


In [40]:
data = get_data_2()
m = 5
print(slice_last_point(data, m)[1])

[[7. 2. 7. 1. 6. 5. 0. 0. 3. 1. 9. 9. 6. 6. 7. 8. 8. 7. 0. 8. 6. 8. 9. 8.
  3. 6. 1. 7. 4. 9. 2. 0. 8. 2. 7. 8. 4. 4. 1. 7.]
 [6. 9. 4. 1. 5. 9. 7. 1. 3. 5. 7. 3. 6. 6. 7. 9. 1. 9. 6. 0. 3. 8. 4. 1.
  4. 5. 0. 3. 1. 4. 4. 4. 0. 0. 8. 4. 6. 9. 3. 3.]
 [2. 1. 2. 1. 3. 4. 1. 1. 0. 7. 8. 4. 3. 5. 6. 3. 2. 9. 8. 1. 4. 0. 8. 3.
  9. 5. 5. 1. 7. 8. 6. 4. 7. 3. 5. 3. 6. 4. 7. 3.]
 [0. 5. 9. 3. 7. 5. 5. 8. 0. 8. 3. 6. 9. 3. 2. 7. 0. 3. 0. 3. 6. 1. 9. 2.
  9. 4. 9. 1. 3. 2. 4. 9. 7. 4. 9. 4. 1. 2. 7. 2.]
 [3. 9. 7. 6. 6. 2. 3. 6. 0. 8. 0. 7. 6. 5. 9. 6. 5. 2. 7. 1. 9. 2. 2. 5.
  6. 4. 2. 2. 1. 0. 9. 0. 2. 8. 3. 0. 8. 8. 1. 0.]]


## Slicing: Fixed Point

In [41]:
def slice_fixed_point(x, s, m):
    """
    Takes one 3-dimensional array with the starting position and the 
    length of the output instances. Your task is to slice the instances 
    from the same starting position for the given length.

    Parameters:
    x (numpy.ndarray): 1-d numpy array with 2-d numpy arrays as elements (n, ?, k).
    s (int): The starting reference index in dimension 2.
    m (int): The cutoff reference index in dimension 2.
    
    Returns:
    numpy.ndarray: A 3-dimensional int numpy array of shape (n, m-s, k)
    """
    spectrograms = x
    
    # Input function dimension specification
    assert(spectrograms.ndim == 1)
    for utter in spectrograms:
        assert(utter.ndim == 2)

    # Pre-define output function dimension specification
    dim1 = spectrograms.shape[0]     # n
    dim2 = m-s                      # m-s
    dim3 = spectrograms[0].shape[1]  # k

    result = np.zeros((dim1,dim2,dim3))

    #### Start of your code ####
    for i in range(dim1):
        result[i,:,:] = x[i][s:m, :]
    
    ####  End of your code  ####

    # Assert output function dimension specification
    assert(result.shape[0] == dim1)
    assert(result.shape[1] == dim2)
    assert(result.shape[2] == dim3)
    
    return result

In [42]:
spectrograms = get_data_1()
start = 0
end = 2
print(slice_fixed_point(spectrograms, start, end))

[[[3. 1. 4. 1.]
  [5. 9. 2. 6.]]

 [[9. 7. 9. 3.]
  [2. 3. 8. 4.]]

 [[6. 2. 6. 4.]
  [3. 3. 8. 3.]]]


In [43]:
data = get_data_2()
s = 2
m = 5
print(slice_fixed_point(data, s, m)[1])

[[8. 7. 0. 3. 8. 7. 7. 1. 8. 4. 7. 0. 4. 9. 0. 6. 4. 2. 4. 6. 3. 3. 7. 8.
  5. 0. 8. 5. 4. 7. 4. 1. 3. 3. 9. 2. 5. 2. 3. 5.]
 [7. 2. 7. 1. 6. 5. 0. 0. 3. 1. 9. 9. 6. 6. 7. 8. 8. 7. 0. 8. 6. 8. 9. 8.
  3. 6. 1. 7. 4. 9. 2. 0. 8. 2. 7. 8. 4. 4. 1. 7.]
 [6. 9. 4. 1. 5. 9. 7. 1. 3. 5. 7. 3. 6. 6. 7. 9. 1. 9. 6. 0. 3. 8. 4. 1.
  4. 5. 0. 3. 1. 4. 4. 4. 0. 0. 8. 4. 6. 9. 3. 3.]]


## Slicing: Random Point

In [44]:
def slice_random_point(x, d):
    """
    Takes one 3-dimensional array with the length of the output instances.
    Your task is to slice the instances from a random point in each of the
    utterances with the given length. Please use offset and refer to their 
    mathematical correspondance.

    Parameters: 
    x (numpy.ndarray): 1-d numpy array with 2-d numpy arrays as elements (n, ?, k).
    d (int): The resulting size of the data in dimension 2.
    
    Returns: 
    numpy.ndarray: A 3-dimensional int numpy array of shape (n, d, k)
    """
    spectrograms = x
    
    # Input function dimension specification
    assert(spectrograms.ndim == 1)
    for utter in spectrograms:
        assert(utter.ndim == 2)
        assert(utter.shape[0] >= d)

    offset = [np.random.randint(utter.shape[0]-d+1)
              if utter.shape[0]-d > 0 else 0
              for utter in spectrograms]

    # Pre-define output function dimension specification
    dim1 = spectrograms.shape[0]    # n
    dim2 = d                       # d
    dim3 = spectrograms[0].shape[1] # k

    result = np.zeros((dim1,dim2,dim3))

    #### Start of your code ####
    for i in range(dim1):
        result[i,:,:] = x[i][offset[i]:offset[i]+d,:]
    
    
    ####  End of your code  ####

    # Assert output function dimension specification
    assert(result.shape[0] == dim1)
    assert(result.shape[1] == dim2)
    assert(result.shape[2] == dim3)
    
    return result

In [45]:
np.random.seed(1)
spectrograms = get_data_1()
duration = 2
print(slice_random_point(spectrograms, duration))

[[[5. 9. 2. 6.]
  [5. 3. 5. 8.]]

 [[9. 7. 9. 3.]
  [2. 3. 8. 4.]]

 [[6. 2. 6. 4.]
  [3. 3. 8. 3.]]]


In [46]:
data = get_data_2()
d = 4
print(slice_random_point(data, d)[1])

[[3. 3. 7. 9. 9. 9. 7. 3. 2. 3. 9. 7. 7. 5. 1. 2. 2. 8. 1. 5. 8. 4. 0. 2.
  5. 5. 0. 8. 1. 1. 0. 3. 8. 8. 4. 4. 0. 9. 3. 7.]
 [3. 2. 1. 1. 2. 1. 4. 2. 5. 5. 5. 2. 5. 7. 7. 6. 1. 6. 7. 2. 3. 1. 9. 5.
  9. 9. 2. 0. 9. 1. 9. 0. 6. 0. 4. 8. 4. 3. 3. 8.]
 [8. 7. 0. 3. 8. 7. 7. 1. 8. 4. 7. 0. 4. 9. 0. 6. 4. 2. 4. 6. 3. 3. 7. 8.
  5. 0. 8. 5. 4. 7. 4. 1. 3. 3. 9. 2. 5. 2. 3. 5.]
 [7. 2. 7. 1. 6. 5. 0. 0. 3. 1. 9. 9. 6. 6. 7. 8. 8. 7. 0. 8. 6. 8. 9. 8.
  3. 6. 1. 7. 4. 9. 2. 0. 8. 2. 7. 8. 4. 4. 1. 7.]]


## Slicing: Ending Pattern

In [52]:
def pad_ending_pattern(x):
    """
    Takes one 3-dimensional array. Your task is to pad the instances from 
    the end position as shown in the example below. That is, you need to 
    pads with the reflection of the vector mirrored along the edge of the array.
    
    Parameters: 
    x (numpy.ndarray): 1-d numpy array with 2-d numpy arrays as elements.
    
    Returns: 
    numpy.ndarray: 3-dimensional int numpy array
    """
    spectrograms = x
    
    # Input function dimension specification
    assert(spectrograms.ndim == 1)
    for utter in spectrograms:
        assert(utter.ndim == 2)

    # Pre-define output function dimension specification
    dim1 = spectrograms.shape[0]    # n
    dim2 = max([utter.shape[0] for utter in spectrograms]) # m
    dim3 = spectrograms[0].shape[1] # k

    result = np.zeros((dim1, dim2, dim3))

    #### Start of your code ####
    for i in range(dim1):
        x_dim2 = x[i].shape[0]
        result[i,:x_dim2,:] = x[i][:,:]
        padded = dim2 - x_dim2
        for j in range(-1, -(padded + 1), -1):
            result[i,x_dim2-j-1,:] = x[i][j,:]
    
    ####  End of your code  ####

    # Assert output function dimension specification
    assert(result.shape[0] == dim1)
    assert(result.shape[1] == dim2)
    assert(result.shape[2] == dim3)
    
    return result

In [53]:
spectrograms = get_data_1()
duration = 2
print(pad_ending_pattern(spectrograms))

[[[3. 1. 4. 1.]
  [5. 9. 2. 6.]
  [5. 3. 5. 8.]
  [5. 3. 5. 8.]]

 [[9. 7. 9. 3.]
  [2. 3. 8. 4.]
  [2. 3. 8. 4.]
  [9. 7. 9. 3.]]

 [[6. 2. 6. 4.]
  [3. 3. 8. 3.]
  [2. 7. 9. 5.]
  [0. 2. 8. 8.]]]


In [59]:
data = get_data_2()
print(pad_ending_pattern(data)[4])

[[8. 2. 4. 3. 1. 6. 5. 8. 4. 3. 6. 5. 3. 7. 8. 8. 3. 7. 8. 5. 7. 2. 7. 8.
  0. 7. 4. 8. 4. 4. 0. 4. 8. 0. 0. 4. 7. 3. 7. 7.]
 [2. 2. 1. 7. 0. 7. 5. 9. 7. 1. 1. 2. 4. 1. 4. 5. 8. 2. 1. 6. 3. 0. 3. 9.
  5. 1. 3. 7. 1. 1. 7. 9. 4. 2. 0. 3. 2. 4. 0. 0.]
 [9. 3. 8. 3. 0. 4. 4. 0. 2. 5. 5. 8. 2. 7. 3. 6. 1. 0. 2. 2. 5. 5. 1. 2.
  8. 7. 3. 7. 3. 1. 0. 1. 0. 8. 8. 5. 3. 3. 1. 0.]
 [6. 1. 6. 9. 5. 7. 0. 1. 4. 9. 5. 1. 6. 5. 4. 4. 4. 7. 2. 2. 6. 5. 3. 0.
  8. 8. 1. 8. 7. 5. 7. 9. 4. 0. 7. 2. 3. 9. 5. 4.]
 [0. 4. 5. 8. 1. 4. 8. 0. 1. 1. 8. 9. 4. 9. 0. 3. 0. 7. 0. 8. 1. 2. 8. 5.
  8. 2. 1. 3. 5. 0. 2. 5. 8. 6. 2. 7. 7. 1. 8. 4.]
 [9. 3. 3. 2. 9. 0. 4. 6. 4. 3. 2. 3. 1. 1. 2. 7. 2. 7. 0. 1. 8. 0. 5. 2.
  8. 0. 4. 0. 3. 8. 1. 6. 4. 6. 9. 6. 4. 7. 2. 9.]
 [9. 3. 3. 2. 9. 0. 4. 6. 4. 3. 2. 3. 1. 1. 2. 7. 2. 7. 0. 1. 8. 0. 5. 2.
  8. 0. 4. 0. 3. 8. 1. 6. 4. 6. 9. 6. 4. 7. 2. 9.]
 [0. 4. 5. 8. 1. 4. 8. 0. 1. 1. 8. 9. 4. 9. 0. 3. 0. 7. 0. 8. 1. 2. 8. 5.
  8. 2. 1. 3. 5. 0. 2. 5. 8. 6. 2. 7. 7. 1. 8. 4.]]

## Padding: Constant Central Pattern

In [56]:
def pad_constant_central_pattern(x, cval):
    """
    Takes one 3-dimensional array with the constant value of padding. 
    Your task is to pad the instances with the given constant value while
    maintaining the array at the center of the padding.

    Parameters: 
    x (numpy.ndarray): 1-d numpy array with 2-d numpy arrays as elements.
    cval (numpy.int64): scalar quantity.
    
    Returns: 
    numpy.ndarray: 3-dimensional int numpy array, (n, m, k).
    """
    spectrograms = x
    
    # Input function dimension specification
    assert(spectrograms.ndim == 1)
    for utter in spectrograms:
        assert(utter.ndim == 2)

    dim1 = spectrograms.shape[0]    # n
    dim2 = max([utter.shape[0] for utter in spectrograms]) # m
    dim3 = spectrograms[0].shape[1] # k

    result = np.ones((dim1,dim2,dim3))

    #### Start of your code ####
    for i in range(dim1):
        result[i,:,:] = cval
        x_dim2 = x[i].shape[0]
        start = (dim2 - x_dim2) // 2
        result[i,start:start+x_dim2,:] = x[i]
    
    ####  End of your code  ####

    # Assert output function dimension specification
    assert(result.shape[0] == dim1)
    assert(result.shape[1] == dim2)
    assert(result.shape[2] == dim3)
    
    return result

In [57]:
spectrograms = get_data_1()
duration = 2
print(pad_constant_central_pattern(spectrograms, 0))

[[[3. 1. 4. 1.]
  [5. 9. 2. 6.]
  [5. 3. 5. 8.]
  [0. 0. 0. 0.]]

 [[0. 0. 0. 0.]
  [9. 7. 9. 3.]
  [2. 3. 8. 4.]
  [0. 0. 0. 0.]]

 [[6. 2. 6. 4.]
  [3. 3. 8. 3.]
  [2. 7. 9. 5.]
  [0. 2. 8. 8.]]]


In [58]:
data = get_data_2()
print(pad_constant_central_pattern(data, cval = 0)[0])

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [3. 5. 2. 4. 7. 6. 8. 8. 1. 6. 7. 7. 8. 1. 5. 9. 8. 9. 4. 3. 0. 3. 5. 0.
  2. 3. 8. 1. 3. 3. 3. 7. 0. 1. 9. 9. 0. 4. 7. 3.]
 [2. 7. 2. 0. 0. 4. 5. 5. 6. 8. 4. 1. 4. 9. 8. 1. 1. 7. 9. 9. 3. 6. 7. 2.
  0. 3. 5. 9. 4. 4. 6. 4. 4. 3. 4. 4. 8. 4. 3. 7.]
 [5. 5. 0. 1. 5. 9. 3. 0. 5. 0. 1. 2. 4. 2. 0. 3. 2. 0. 7. 5. 9. 0. 2. 7.
  2. 9. 2. 3. 3. 2. 3. 4. 1. 2. 9. 1. 4. 6. 8. 2.]
 [3. 0. 0. 6. 0. 6. 3. 3. 8. 8. 8. 2. 3. 2. 0. 8. 8. 3. 8. 2. 8. 4. 3. 0.
  4. 3. 6. 9. 8. 0. 8. 5. 9. 0. 9. 6. 5. 3. 1. 8.]
 [0. 4. 9. 6. 5. 7. 8. 8. 9. 2. 8. 6. 6. 9. 1. 6. 8. 8. 3. 2. 3. 6. 3. 6.
  5. 7. 0. 8. 4. 6. 5. 8. 2. 3. 9. 7. 5. 3. 4. 5.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]

# PyTorch

In [60]:
def numpy2tensor(x):
    return torch.from_numpy(x)

In [61]:
X = np.random.randint(-1000, 1000, size=3000)

print(type(numpy2tensor(X)))

<class 'torch.Tensor'>


In [62]:
def tensor2numpy(x):
    return x.numpy()

In [63]:
X = np.random.randint(-1000, 1000, size=3000)
X = torch.from_numpy(X)

print(type(tensor2numpy(X)))

<class 'numpy.ndarray'>


In [66]:
def tensor_sumproducts(x, y):
    return (x.sum() * y).sum()

In [67]:
np.random.seed(0)
X = np.random.randint(-1000, 1000, size=3000)
X = torch.from_numpy(X)
Y = np.random.randint(-1000, 1000, size=3000)
Y = torch.from_numpy(Y)

print(tensor_sumproducts(X,Y))

tensor(265421520)


In [72]:
def tensor_ReLU(x):
    zero = torch.tensor([0])
    return torch.where(x >= 0, x, zero)

In [73]:
np.random.seed(0)
X = np.random.randint(-1000, 1000, size=(1000,1000))
X = torch.from_numpy(X)

print(tensor_ReLU(X))

tensor([[  0,   0, 653,  ...,   0,   0,   0],
        [  0,   0,   0,  ..., 988,   0,   0],
        [265,   0, 608,  ..., 773, 961,   0],
        ...,
        [429, 102,   0,  ..., 467, 118,   0],
        [532,  55,   0,  ..., 912, 779, 294],
        [  0,  51,   0,  ...,   0,   0,   0]])


In [74]:
def tensor_ReLU_prime(x):
    ones = torch.tensor([1])
    zero = torch.tensor([0])
    return torch.where(x >= 0, ones, zero)

In [75]:
np.random.seed(0)
X = np.random.randint(-1000, 1000, size=(1000,1000))
X = torch.from_numpy(X)

print(tensor_ReLU_prime(X))

tensor([[0, 0, 1,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 1, 0, 0],
        [1, 0, 1,  ..., 1, 1, 0],
        ...,
        [1, 1, 0,  ..., 1, 1, 0],
        [1, 1, 0,  ..., 1, 1, 1],
        [0, 1, 0,  ..., 0, 0, 0]])
